In [3]:
# TO BE RUN ONLY ONCE!
import os
os.chdir("../")

In [2]:
## *******************************************************************************

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import subprocess
import pymc3 as pm
import seaborn as sns

import spatial_mix.hdp_utils as hdp_utils
from spatial_mix.utils import *
from spatial_mix.protos.py.univariate_mixture_state_pb2 import UnivariateState, UnivariateMixtureState, HdpState

current_palette = sns.color_palette()

## Simulate data

<img src="simulation_scenario.png">

Simulate data as follows: for each little square we simulate data from a 3 components mixtures
$$ y_{ij} \sim w_{i1} N(-5, 1) + w_{i2} N(0, 1) + w_{i3} N(5, 1)$$

the weights depend on the location (x, y) of the center of the square (the orange dot)
$$ \widetilde{w}_{il} = \alpha_l x_i + \beta_l y_i \quad l=1, 2$$

finally $w_i = alr^{-1} ([\widetilde{w}_{i1}, \widetilde{w}_{i2}])$

In [4]:
def inv_alr(x):
    out = np.exp(np.hstack((x, 1)))
    return out / np.sum(out)

In [5]:
def simulate_from_mixture(weights):
    means = [-5, 0, 5]
    comp = np.random.choice(3, p=weights)
    return np.random.normal(loc=means[comp], scale=1)

In [6]:
centers = np.array([[0, 1], [1, 1], [2, 1], [3, 1], [0, 0], [1, 0], [2, 0], [3, 0]])
centers = centers / 10
alpha1 = 0.3
alpha2 = -0.3
beta1 = 0.3
beta2 = -0.3

weights = []
for center in centers:
    w1 = alpha1 * center[0] + beta1 * center[1]
    w2 = alpha2 * center[0] + beta2 * center[1]
    weights.append(inv_alr([w1, w2]))

In [ ]:
Dmat1 = np.zeros((8, 8))

for i in range(7):
    Dmat1[i, i+1] = 1
    Dmat1[i+1, i] = 1
    if (i < 4):
        Dmat1[i, i+4] = 1
        Dmat1[i+4, i] = 1
    
print(Dmat1)

In [ ]:
np.savetxt("data/simulated_D1.csv", Dmat1, delimiter=",")

In [ ]:
data = []
for i in range(8):
    numSamples = 100
    if i == 5:
        numSamples = 20
        
    for j in range(numSamples):
        data.append([i, simulate_from_mixture(weights[i])])

In [ ]:
df = pd.DataFrame(data, columns=["group", "data"])
df.to_csv("data/simulated_data1.csv", index=False)

# Simulated Scenario 2

In [7]:
Dmat2 = np.array([
    [0, 1, 1, 1, 0, 0, 0, 0],
    [1, 0, 1, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 1, 0, 1, 1],
    [0, 0, 0, 0, 1, 1, 0, 1],
    [0, 0, 0, 0, 1, 1, 1, 0]
]) 

print(Dmat2)

[[0 1 1 1 0 0 0 0]
 [1 0 1 1 0 0 0 0]
 [1 1 0 1 0 0 0 0]
 [1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 1]
 [0 0 0 0 1 0 1 1]
 [0 0 0 0 1 1 0 1]
 [0 0 0 0 1 1 1 0]]


In [8]:
w_group1 = np.array([0.25, 0.25, 0.5])
w_group2 = np.array([0.4, 0.4, 0.2])

weights = []

data = []
for i in range(8):
    w = w_group1 if i < 4 else w_group2
    weights.append(w)

In [9]:
data = []
for i in range(8):
    numSamples = 1000
#     if i == 5:
#         numSamples = 20

    for j in range(numSamples):
        data.append([i, simulate_from_mixture(weights[i])])
        
# df = pd.DataFrame(data, columns=["group", "data"])
# df.to_csv("data/simulated_data2.csv", index=False)
# np.savetxt("data/simulated_D2.csv", Dmat2, delimiter=",")

# call C++ from terminal

In [ ]:
subprocess.call(
    "./spatial_mix/run_from_file.out"
    " data/simulated_data2.csv"
    " data/simulated_D2.csv"
    " data/chains_simulated2.recordio".split())

In [ ]:
print("./spatial_mix/run_from_file.out"
    " data/simulated_data2.csv"
    " data/simulated_D2.csv"
    " data/chains_simulated2.recordio")

## Run via the Python interface

In [40]:
from spatial_mix.protos.py.sampler_params_pb2 import SamplerParams
from google.protobuf import text_format

df = pd.read_csv("./spatial_mix/data_simulation_1.csv")

burnin = 10000
niter = 10000
thin = 10


In [41]:
df['Data']

0       -2.022690
1       -1.004620
2       -3.468810
3       -3.939700
4       -2.073080
5       -2.037240
6       -1.624360
7       -1.936110
8       -1.395200
9       -2.315390
10      -2.897630
11      -1.456780
12      -0.026193
13       1.123770
14      -1.248010
15      -2.138100
16      -2.200130
17      -2.054780
18      -0.870412
19      -2.444050
20       0.017322
21      -2.889110
22      -1.395890
23      -0.626187
24      -2.663320
25      -2.047290
26      -1.565470
27      -2.707800
28      -1.916950
29      -1.310920
          ...    
4970    -2.044030
4971     5.880630
4972    -1.404940
4973     2.140500
4974    -2.656720
4975     0.102388
4976     0.338712
4977    -2.202290
4978    -0.799216
4979     0.315849
4980     0.498536
4981   -23.319200
4982     1.603000
4983    -1.860510
4984     0.471645
4985    -0.188141
4986     0.285361
4987   -10.699200
4988   -28.484400
4989     0.673829
4990   -19.842100
4991    -1.247690
4992    -6.878190
4993     0.228397
4994    -0

In [50]:
numGroups = max(df['Group'])+1
datas = []
for g in range(numGroups):
    datas.append(df[df['Group'] == g]['Data'].values)


In [51]:
W = np.ones((numGroups,numGroups))-np.identity(numGroups)
W

array([[0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 1., 0.]])

In [ ]:
chains = runSpatialMixtureSampler(burnin, niter, thin, W,
                                  "spatial_mix/resources/sampler_params.asciipb",
                                  datas, [])

Burn-in, iter # 200  /  10000
Burn-in, iter # 400  /  10000


## Plot density estimates

In [ ]:
df = pd.read_csv("data/data_simulation_1.csv")
#chains = loadChains("data/chains_simulated2.recordio")

In [ ]:
# datas = []
# for g in range(8):
#     datas.append(df[df.group == g].data.to_list())

In [ ]:
# chains 

In [11]:
xgrid = np.linspace(-10, 10, 1000)
dens = estimateDensities(chains, xgrid)

In [15]:
g = 5
true_d = weights[g][0] * norm.pdf(xgrid, -5, 1.0) + \
         weights[g][1] * norm.pdf(xgrid, 0.0, 1.0) + \
         weights[g][2] * norm.pdf(xgrid, 5.0, 1.0)

hell_dists = post_hellinger_dist(dens[g], true_d, xgrid)

TypeError: map() must have at least two arguments.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 8))
axes = axes.flat

for g in range(8):
    axes[g].plot(xgrid, 
                 weights[g][0] * norm.pdf(xgrid, -5, 1.0) +
                 weights[g][1] * norm.pdf(xgrid, 0.0, 1.0) +
                 weights[g][2] * norm.pdf(xgrid, 5.0, 1.0))
    axes[g].plot(xgrid, np.mean(dens[g], 0))
    axes[g].set_ylim([0, 0.3])
    intervals = np.array([pm.stats.hpd(dens[g][:, i], 0.05) for i in range(dens[g].shape[1])])
    axes[g].fill_between(xgrid, intervals[:, 0], intervals[:, 1], alpha=0.3, color=current_palette[2])
    axes[g].set_title("Group: {0}".format(g))
#     sns.kdeplot(datas[g], ax=axes[g], color=current_palette[3])
    
plt.tight_layout()

In [ ]:
estimateds = estimateDensities(chains, datas)
lpml(estimateds)

In [ ]:
chains[0].rho

In [ ]:
## Plot RhoChain
rhoChain = list(map(lambda x: x.rho, chains))
plt.plot(range(len(chains)), rhoChain)

In [ ]:
Wmat = Dmat2.astype(np.double)

## Hdp

In [ ]:
subprocess.call(
    "./spatial_mix/run_hdp_from_file.out"
    " data/simulated_data2.csv"
    " data/chains_hdp_simulated2.recordio".split())

In [ ]:
hdpChains = loadChains("data/chains_hdp_simulated2.recordio", HdpState)

In [ ]:
xgrid = np.linspace(-10, 10, 1000)
densHdp = hdp_utils.estimateDensities(hdpChains, xgrid)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 8))
axes = axes.flat

for g in range(8):
    axes[g].plot(xgrid, 
                 weights[g][0] * norm.pdf(xgrid, -5, 1.0) +
                 weights[g][1] * norm.pdf(xgrid, 0.0, 1.0) +
                 weights[g][2] * norm.pdf(xgrid, 5.0, 1.0))
    obs = df[df["group"] == g]["data"].values
    # sns.kdeplot(obs, ax=axes[g])
    axes[g].set_ylim([0, 0.3])
    axes[g].plot(xgrid, np.mean(dens[g], 0))
    intervals = np.array([pm.stats.hpd(densHdp[g][:, i], 0.05) for i in range(densHdp[g].shape[1])])
    axes[g].fill_between(xgrid, intervals[:, 0], intervals[:, 1], alpha=0.3, color=current_palette[2])
    axes[g].set_title("Group: {0}".format(g))
    
plt.tight_layout()

In [ ]:
estimatedHdp = hdp_utils.estimateDensities(hdpChains, datas)
lpml(estimatedHdp)

In [ ]:
lpml(estimatedHdp[2])

In [ ]:
lpml(estimateds[2])